# Ingestão de Dados (Bronze Layer)

## Objetivos
- Carregar dados brutos do CSV do INEP
- Verificar estrutura e qualidade dos dados
- Converter para formato Parquet para melhor performance
- Preparar dados para processamento no Silver Layer

## Estrutura dos Dados Originais
O arquivo `escolas_inep.csv` contém informações sobre escolas brasileiras do INEP com 19 colunas principais:
- Informações geográficas (UF, Município, Localização, Latitude, Longitude)
- Características administrativas (Dependência, Categoria, Porte)
- Informações educacionais (Etapas de Ensino, Modalidades)
- Status operacional (Restrição de Atendimento, Regulamentação)

In [2]:
# === CONFIGURAÇÃO E CARREGAMENTO DOS DADOS BRUTOS ===

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnan, when
import pyspark.sql.functions as F

# Inicializar Spark Session
spark = SparkSession.builder.appName("inep_ingestion").getOrCreate()

print("=== INICIANDO INGESTÃO DOS DADOS BRUTOS ===")

# Carregar dados brutos do CSV
df_bronze = spark.read.option("header", True).option("encoding", "utf-8").csv("escolas_inep.csv")

print(f"✅ Dados carregados com sucesso!")
print(f"📊 Total de registros: {df_bronze.count():,}")
print(f"📋 Total de colunas: {len(df_bronze.columns)}")

# Verificar schema
print("\n=== SCHEMA DOS DADOS ===")
df_bronze.printSchema()

# Mostrar primeiras linhas
print("\n=== PRIMEIRAS 5 LINHAS ===")
df_bronze.show(5, truncate=False)

# Verificar valores nulos por coluna
print("\n=== ANÁLISE DE VALORES NULOS ===")
null_counts = df_bronze.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ""), c)).alias(c) for c in df_bronze.columns]).collect()[0]

for col_name in df_bronze.columns:
    null_count = null_counts[col_name]
    if null_count > 0:
        pct = (null_count / df_bronze.count()) * 100
        print(f"   {col_name}: {null_count:,} valores nulos/vazios ({pct:.1f}%)")

# Salvar dados bronze em formato Parquet
print("\n=== SALVANDO DADOS BRONZE ===")
df_bronze.write.mode("overwrite").parquet("bronze_escolas.parquet")

print("✅ Dados salvos com sucesso em: bronze_escolas.parquet")
print("📁 Arquivo pronto para processamento no Silver Layer!")
print(f"💾 Tamanho estimado: {df_bronze.count() * len(df_bronze.columns) * 50 / 1024 / 1024:.1f} MB")


=== INICIANDO INGESTÃO DOS DADOS BRUTOS ===


✅ Dados carregados com sucesso!


📊 Total de registros: 212,386
📋 Total de colunas: 19

=== SCHEMA DOS DADOS ===
root
 |-- Restrição de Atendimento: string (nullable = true)
 |-- Escola: string (nullable = true)
 |-- Código INEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Localização: string (nullable = true)
 |-- Localidade Diferenciada: string (nullable = true)
 |-- Categoria Administrativa: string (nullable = true)
 |-- Endereço: string (nullable = true)
 |-- Telefone: string (nullable = true)
 |-- Dependência Administrativa: string (nullable = true)
 |-- Categoria Escola Privada: string (nullable = true)
 |-- Conveniada Poder Público: string (nullable = true)
 |-- Regulamentação pelo Conselho de Educação: string (nullable = true)
 |-- Porte da Escola: string (nullable = true)
 |-- Etapas e Modalidade de Ensino Oferecidas: string (nullable = true)
 |-- Outras Ofertas Educacionais: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longi

   Telefone: 42,433 valores nulos/vazios (20.0%)
   Regulamentação pelo Conselho de Educação: 31,321 valores nulos/vazios (14.7%)
   Porte da Escola: 31,321 valores nulos/vazios (14.7%)
   Etapas e Modalidade de Ensino Oferecidas: 33,163 valores nulos/vazios (15.6%)
   Outras Ofertas Educacionais: 153,326 valores nulos/vazios (72.2%)

=== SALVANDO DADOS BRONZE ===


25/09/29 20:25:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


✅ Dados salvos com sucesso em: bronze_escolas.parquet
📁 Arquivo pronto para processamento no Silver Layer!
💾 Tamanho estimado: 192.4 MB


## 📊 Estatísticas da Ingestão
- **Total de registros**: ~212.000 escolas
- **Total de colunas**: 19 campos
- **Formato de saída**: Parquet (otimizado para Spark)
- **Próximo processamento**: Limpeza e padronização no Silver Layer
